In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
import pandas as pd

In [2]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 15000
data_path = './fra-eng/fra.txt'

In [3]:
# Vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[  min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
len(input_characters)

74

In [5]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [6]:
len(input_texts), num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length

(15000, 74, 98, 16, 59)

In [7]:
input_token_index = dict([(char,i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i, char in enumerate(target_characters)])

In [8]:
# input_token_index

In [33]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype = 'float32'
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32'
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32'
)

In [34]:
# One hot representation
for i, (input_text, target_text) in enumerate(zip(input_texts,target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1: ,input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1 ,target_token_index[char]] = 1.
        
        decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
        decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [37]:
encoder_input_data[0][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)

In [38]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_state = [state_h, state_c]

In [39]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences = True, return_state=True)
decoder_outputs , _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_state)
decoder_dense = Dense(num_decoder_tokens , activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [40]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data, 
          batch_size=batch_size, epochs=epochs, validation_split=0.25)

Epoch 1/100
176/176 [==============================] - 6s 24ms/step - loss: 2.0807 - accuracy: 0.9778 - val_loss: 2.4058 - val_accuracy: 0.9831
Epoch 2/100
176/176 [==============================] - 4s 21ms/step - loss: 2.1927 - accuracy: 0.9830 - val_loss: 2.6146 - val_accuracy: 0.9831
Epoch 3/100
176/176 [==============================] - 4s 21ms/step - loss: 2.3875 - accuracy: 0.9831 - val_loss: 2.8207 - val_accuracy: 0.9831
Epoch 4/100
176/176 [==============================] - 4s 21ms/step - loss: 2.5819 - accuracy: 0.9831 - val_loss: 3.0541 - val_accuracy: 0.9831
Epoch 5/100
176/176 [==============================] - 4s 21ms/step - loss: 2.7733 - accuracy: 0.9831 - val_loss: 3.2418 - val_accuracy: 0.9831
Epoch 6/100
176/176 [==============================] - 4s 22ms/step - loss: 2.9633 - accuracy: 0.9831 - val_loss: 3.4633 - val_accuracy: 0.9831
Epoch 7/100
176/176 [==============================] - 4s 21ms/step - loss: 3.1693 - accuracy: 0.9831 - val_loss: 3.6991 - val_accuracy:

176/176 [==============================] - 4s 22ms/step - loss: 14.0563 - accuracy: 0.9831 - val_loss: 15.8760 - val_accuracy: 0.9831
Epoch 58/100
176/176 [==============================] - 4s 22ms/step - loss: 14.2962 - accuracy: 0.9831 - val_loss: 16.1422 - val_accuracy: 0.9831
Epoch 59/100
176/176 [==============================] - 4s 22ms/step - loss: 14.5307 - accuracy: 0.9831 - val_loss: 16.4171 - val_accuracy: 0.9831
Epoch 60/100
176/176 [==============================] - 4s 22ms/step - loss: 14.7664 - accuracy: 0.9831 - val_loss: 16.6765 - val_accuracy: 0.9831
Epoch 61/100
176/176 [==============================] - 4s 22ms/step - loss: 15.0116 - accuracy: 0.9831 - val_loss: 16.9557 - val_accuracy: 0.9831
Epoch 62/100
176/176 [==============================] - 4s 22ms/step - loss: 15.2551 - accuracy: 0.9831 - val_loss: 17.2204 - val_accuracy: 0.9831
Epoch 63/100
176/176 [==============================] - 4s 22ms/step - loss: 15.4945 - accuracy: 0.9831 - val_loss: 17.5018 - val_a

In [41]:
# model.save('Translation tutorial - Encoder and Decoder.h5', model)